In [ ]:
import openai
from llm_tale.llm_planners import prompt_folder
from llm_tale.llm_planners.utils.plan_utils import (
    prompt_task_planner,
    prompt_affordance_identifier,
    prompt_affordance_planner,
    prompt_code_translator,
)

openai.api_key = "YOUR_API_KEY"  # Replace with your OpenAI API key

In [2]:
task = "PegInsert"
model = "gpt-4o"

if task == "PegInsert":
    from llm_tale.envs.maniskill_envs.tasks.PegInsert import Instruction, Objects
elif task == "StackCube":
    from llm_tale.envs.maniskill_envs.tasks.StackCube import Instruction, Objects
elif task == "PickCube":
    from llm_tale.envs.maniskill_envs.tasks.PickCube import Instruction, Objects
elif task == "OpenDrawer":
    from llm_tale.envs.rlbench_envs.tasks.OpenDrawer import Instruction, Objects
elif task == "PutBox":
    from llm_tale.envs.rlbench_envs.tasks.PutBox import Instruction, Objects
elif task == "TakeLid":
    from llm_tale.envs.rlbench_envs.tasks.TakeLid import Instruction, Objects

print("Task: ", task)
print("Instruction: ", Instruction)
print(Objects)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/jelle/llm_tale/.venv/lib/python3.10/site-packages/sapien/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Task:  PegInsert
Instruction:  insert the peg into the hole

Objects:
    Peg = Object( # a peg is laying on the ground
        position, 
        orientation, # x-axis as center axis
        attributes=[head, end])
    Hole = Object(
        position, 
        orientation, # x-axis as center axis
        attributes=None) 



In [3]:
# Task planner
assert openai.api_key != "YOUR_API_KEY", "Please set your OpenAI API key to generate new plans."

res = {}
response = prompt_task_planner(Instruction, Objects, prompt_folder, verbose=False, model=model)
task_plan = "{" + response.text.split("{")[1].split("}")[0] + "}"
res["task_plan"] = eval(task_plan)
code = prompt_code_translator(task_plan, Objects, prompt_folder, verbose=False, model=model)
_res = "{" + code.text.split("{")[1].split("}")[0] + "}"
res["code"] = eval(_res)

print(res["code"])
print(res["task_plan"])

{'1': 'robot.pick(Peg)', '2': "robot.transport(Peg, Hole, 'INSERT')"}
{'1': 'Pick up the peg.', '2': 'Insert the peg into the hole.'}


In [4]:
# Affordance Identifier
assert openai.api_key != "YOUR_API_KEY", "Please set your OpenAI API key to generate new plans."

aff_space = {}
for idx, _step in enumerate(res["task_plan"]):
    step = res["task_plan"][_step]
    response = prompt_affordance_identifier(step, Objects, prompt_folder, verbose=False, model=model)
    _res = "{" + response.text.split("{")[1].split("}")[0] + "}"
    _res = eval(_res)
    print(response.text)
    if idx == 0:
        aff_space["pick"] = _res
    elif idx == 1:
        aff_space["place"] = _res
res["aff_space"] = aff_space
pick = aff_space["pick"]

OUTPUT Comments:
    Position variation: 2. There are two options (head, end) for picking position.
    Orientation variation: None. The peg is lying on the ground, so the robot can only pick it from the top.

OUTPUT:
{
    "1": "pick the peg from the head",
    "2": "pick the peg from the end",
}
OUTPUT Comments:
    Position variation: None, the head of the peg should be aligned with the hole.
    Orientation variation: None, the peg's center axis should be aligned with the hole's center axis for insertion.

OUTPUT:
{
    "1": "insert the peg by aligning its center axis with the hole's center axis, head position should be aligned with the hole position",
}


In [5]:
# Affordance Planner
assert openai.api_key != "YOUR_API_KEY", "Please set your OpenAI API key to generate new plans."

affordance = {}
for k, v in pick.items():
    step = v
    response = prompt_affordance_planner(step, "pick", Objects, prompt_folder, verbose=False, model=model)
    _res = "{" + response.text.split("{")[1].split("}")[0] + "}"
    print(v)
    print(response.text)
    affordance[step] = eval(_res)
if "place" in aff_space:
    place = aff_space["place"]
    for k, v in place.items():
        step = v
        response = prompt_affordance_planner(step, "transport", Objects, prompt_folder, verbose=False, model=model)
        _res = "{" + response.text.split("{")[1].split("}")[0] + "}"
        print(v)
        print(response.text)
        affordance[step] = eval(_res)
res["affordance"] = affordance

pick the peg from the head
Comments: The peg is lying on the ground, so the robot can only pick it from the top. In this case, we choose ABSOLUTE_VERTICAL as the z-axis for the gripper. Since the peg is a cylindrical object, we choose the major_axis as the x-axis for the gripper.

RESPONSE:
{
    "gripper position": "Peg.head",
    "gripper ori z_axis": "ABSOLUTE_VERTICAL",
    "gripper ori x_axis": "Peg.major_axis"
}
pick the peg from the end
Comments: The peg is lying on the ground, so the robot can only pick it from the top. In this case, we choose ABSOLUTE_VERTICAL as the z-axis for the gripper. Since the peg is a cylindrical object, we choose the major_axis as the x-axis for the gripper.

RESPONSE:
{
    "gripper position": "Peg.end",
    "gripper ori z_axis": "ABSOLUTE_VERTICAL",
    "gripper ori x_axis": "Peg.major_axis"
}
insert the peg by aligning its center axis with the hole's center axis, head position should be aligned with the hole position
{
    # Comments
    # The task

In [6]:
import pickle

# Save plan
save = False  # True to save generated plans

if save:
    path = f"../cache/{task.lower()}_code_{model}.pkl"
    print(f"Saving to {path}")
    with open(path, "wb") as f:
        pickle.dump(res, f)

In [7]:
import ipywidgets as widgets
import pickle
from IPython.display import display, clear_output

model = "gpt-4o"


def show_cache(task):
    clear_output(wait=True)

    if task == "PegInsert":
        from llm_tale.envs.maniskill_envs.tasks.PegInsert import Instruction, Objects
    elif task == "StackCube":
        from llm_tale.envs.maniskill_envs.tasks.StackCube import Instruction, Objects
    elif task == "PickCube":
        from llm_tale.envs.maniskill_envs.tasks.PickCube import Instruction, Objects
    elif task == "OpenDrawer":
        from llm_tale.envs.rlbench_envs.tasks.OpenDrawer import Instruction, Objects
    elif task == "PutBox":
        from llm_tale.envs.rlbench_envs.tasks.PutBox import Instruction, Objects
    elif task == "TakeLid":
        from llm_tale.envs.rlbench_envs.tasks.TakeLid import Instruction, Objects

    print("Instruction:", Instruction)
    print("Objects:", Objects)

    path = f"../cache/{task.lower()}_code_{model}.pkl"
    try:
        with open(path, "rb") as f:
            res = pickle.load(f)
        for k, v in res["affordance"].items():
            print(k)
            print(v)
    except FileNotFoundError:
        print(f"File not found: {path}")


def create_task_interface(tasks):
    task_selector = widgets.Dropdown(
        options=tasks,
        description="Select Task:",
    )

    show_button = widgets.Button(description="Show Cache", icon="check")
    output_area = widgets.Output()

    def on_show_button_clicked(b):
        with output_area:
            show_cache(task_selector.value)

    show_button.on_click(on_show_button_clicked)
    display(task_selector, show_button, output_area)


# Call the function with the list of tasks
create_task_interface(["PegInsert", "StackCube", "PickCube", "OpenDrawer", "PutBox", "TakeLid"])

Dropdown(description='Select Task:', options=('PegInsert', 'StackCube', 'PickCube', 'OpenDrawer', 'PutBox', 'T…

Button(description='Show Cache', icon='check', style=ButtonStyle())

Output()